In [147]:
import requests
from bs4 import BeautifulSoup

In [148]:
#inspect page where the list of books are with corresponding links
book_page_list = 'https://www.gutenberg.org/browse/scores/top#books-last30'
html_page = requests.get(book_page_list) 
soup = BeautifulSoup(html_page.content, 'html.parser')

In [150]:
books_list = soup.find('h2', id='books-last30').next_sibling.next_sibling
books_list = books_list.find_all('a')
books_list[:5]

[<a href="/ebooks/1342">Pride and Prejudice by Jane Austen (25923)</a>,
 <a href="/ebooks/84">Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley (15305)</a>,
 <a href="/ebooks/11">Alice's Adventures in Wonderland by Lewis Carroll (14342)</a>,
 <a href="/ebooks/1661">The Adventures of Sherlock Holmes by Arthur Conan Doyle (13114)</a>,
 <a href="/ebooks/98">A Tale of Two Cities by Charles Dickens (11776)</a>]

In [152]:
book_names = []
base_link_url = 'https://www.gutenberg.org'
for book_info in books_list:
    #book name
    book_name = book_info.text.split(' (')[0].replace('  ', ' ')
    book_name = book_name.replace(' ', '_')
    #book link to multiple reading formats i.e. txt or pdf
    book_download_link = base_link_url + str(book_info['href'])
    
    #access new page
    html_page = requests.get(book_download_link) 
    book_soup = BeautifulSoup(html_page.content, 'html.parser')
    
    #get to download options table
    book_download_list = book_soup.find_all('tr', class_='even')
    for d in book_download_list:
        #find content which starts with the id = 'text/plain'
        d_link = d.find(content=lambda value: value and value.startswith("text/plain"))
        #check if the text file exists
        if d_link:
            book_txt_link = d_link.next_element.get('href')
            #links start with '//' or '/' - get rid of it
            if book_txt_link.startswith('//'):
                book_txt_link = book_txt_link[2:]
            elif book_txt_link.startswith('/'):
                book_txt_link = book_txt_link[1:]
            #make website into a link
            book_txt_link = 'https://' + book_txt_link
#             print(book_txt_link)
            
            #record the book if succesfully scrapped
            book_names.append(book_name)
            print(book_name)
            
            #Access book txt files
            txt_page = requests.get(book_txt_link)
            txt_data = txt_page.text

            #save book to txt on local - create and write into new txt file
            f = open('../project_books/'+book_name+'.txt', 'w+')
            f.write(txt_data)
            f.close()

Pride_and_Prejudice_by_Jane_Austen
Frankenstein;_Or,_The_Modern_Prometheus_by_Mary_Wollstonecraft_Shelley
Alice's_Adventures_in_Wonderland_by_Lewis_Carroll
The_Adventures_of_Sherlock_Holmes_by_Arthur_Conan_Doyle
A_Tale_of_Two_Cities_by_Charles_Dickens
The_Importance_of_Being_Earnest:_A_Trivial_Comedy_for_Serious_People_by_Oscar_Wilde
Dracula_by_Bram_Stoker
The_Strange_Case_of_Dr._Jekyll_and_Mr._Hyde_by_Robert_Louis_Stevenson
Moby_Dick;_Or,_The_Whale_by_Herman_Melville
War_and_Peace_by_graf_Leo_Tolstoy
The_Adventures_of_Tom_Sawyer_by_Mark_Twain
The_Romance_of_Lust:_A_classic_Victorian_erotic_novel_by_Anonymous
Ulysses_by_James_Joyce
Et_dukkehjem._English_by_Henrik_Ibsen
The_Picture_of_Dorian_Gray_by_Oscar_Wilde
Beowulf:_An_Anglo-Saxon_Epic_Poem
Jane_Eyre:_An_Autobiography_by_Charlotte_Brontë
Metamorphosis_by_Franz_Kafka
Adventures_of_Huckleberry_Finn_by_Mark_Twain
Great_Expectations_by_Charles_Dickens
The_Yellow_Wallpaper_by_Charlotte_Perkins_Gilman
The_Count_of_Monte_Cristo,_Illustrate

In [154]:
len(book_names)

96